In [46]:
%%html
<style>
.toc-item > li {
    list-style-type: upper-alpha;
}
</style>

## Kamal Shaham         
## D211: Advanced Data Acquisition

<div>
    <h2>Table of Contents</h2>
    <ul class="toc-item">
        <li><a href="#datasets">Dashboard Datasets</a>
            <ul>
                <li><a href="#datasets">Dashboard Datasets</a></li>
                <li><a href="#installation">Dashboard Installation Steps</a></li>
                <li><a href="#usage">Dashboard Usage</a></li>
                <li><a href="#sql">Dashboard SQL</a></li>
            </ul>
        </li>
        <li><a href="#panopto">Dashboard Presentation - Panopto</a>
        </li>
        <li><a href="#dashboard-purpose">Dashboard Report</a>
        <ul>
                <li><a href="#dashboard-purpose">Dashboard Purpose Alignment</a></li>
                <li><a href="#dashboard-justification">Dashboard Justification</a></li>
                <li><a href="#data-preparation">Data Preparation</a></li>
            <li><a href="#dashboard-creation">Dashboard Creation</a></li>
            <li><a href="#data-analysis">Data Analysis Results</a></li>
            <li><a href="#limitations">Limitations</a></li>
            </ul>
        </li>
        <li><a href="#thirdparty">Third-party code references</a></li>
        <li><a href="#references">References</a></li>
    </ul>
</div>

## A. Dashboard Datasets <a id="datasets"></a>

As in the previous class, this analysis will utilize:
- [The WGU medical dataset](https://access.wgu.edu/ASP3/aap/content/ds0fh43lkfd9tf85kvmd.zip)
- [2017-2018 National Health & Nutrition Examination Survey from the Centers for Disease Control and Prevention (CDC)](https://www.kaggle.com/datasets/rileyzurrin/national-health-and-nutrition-exam-survey-2017-2018).

Throughout this data analysis, we refer to the WGU medical dataset as WGU patients and the NHANES dataset as NHS patients.

## A2. Dashboard Installation Steps <a id="installation"></a>

The dashboard is attached to this submission within the D211.zip folder as D211wb.twbx and can be opened using the Virtual Machine. 

1 - Extract the contents from the zip file attached to the submission to 'C:\Users\Public\Downloads' in the VM

2 - To modify the datasets, double-click PgAdmin from the desktop of the VM

3 - In the left menu of PgAdmin, navigate to Server -> Databases -> medical_data

4 - Right-click the medical_data database and select 'Query tool'

5 - In the query menu, select the second icon with a folder called 'Open from File'

6 - Select the queries.sql file located in the 'C:\Users\Public\Downloads' or paste the queries included in section A4

7 - Once the file is imported or the commands are pasted into the Query tool, click the Play icon to run the modification/creation queries

8 - When the queries have completed, close PgAdmin and double-click Tableau 2021.4 from the VM Desktop

9 - Once Tableau 2021.4 has opened, select Open A Workbook and select the file located at C:\Users\Public\Downloads\D211wb.twbx

10 - When prompted to sign-in to Postgres - enter Passw0rd! for the password field

The workbook should now be fully installed and ready to use for this analysis.

### A3. Dashboard Usage <a id="usage"></a>

Once the Tableau dashboard has opened, navigate to the Patient Analysis story tab located in the bottom menu. There are two tabs within this story: one for patient demographics and the other for condition metrics.

The Patient Demographics tab provides visualizations about WGU and NHANES patients, including patient ages, counts of children, and gender breakdowns. These visualizations can be filtered using the age or gender filters located at the top of the tab. By modifying these filters, the visualizations will update to display only the specified criteria. For example, setting the age filter to the 30-40 range will show patient demographics for this selected age group. The same applies to the gender filter. The visualizations themselves also have built-in filters: selecting any cell within a visualization will filter the rest of the visualizations based on the criteria of the selected cell. For example, selecting the male portion of the pie chart will filter the rest of the visualizations to show only male patient demographics. Deselecting the cell will revert the visualizations to their original state.

The Condition Metrics tab provides interactive visualizations for each critical medical condition. The condition rates for both WGU clinic and NHANES patients are displayed to the left of the heatmaps as key performance indicators (KPIs). These metrics show the exact breakdown of each condition and can also be filtered using the age and gender filters. The condition breakdowns by 5-year age groups are shown as heatmaps to the right of each KPI. It was found that the WGU dataset is almost twice as large as the NHANES results, which could affect our analysis. The visualizations on this dashboard are also interactive: selecting any cell within a visualization will filter the rest based on the cell criteria. Deselecting the cell will return the visualizations to their original state.

### A4. Dashboard SQL <a id="sql"></a>

```
-- From looking at the distinct values of gender column, we will rename 'Prefer not to answer' to 'Non-Binary'
UPDATE public.patient
SET gender = 'Non-Binary'
WHERE gender NOT IN ('Male', 'Female', 'Non-Binary');

-- Convert arthritis column to boolean using CAST
ALTER TABLE public.servicesaddon
  ALTER COLUMN arthritis TYPE bool USING CAST((arthritis != 'No') AS bool),
  ALTER COLUMN arthritis SET DEFAULT FALSE;

-- Convert diabetes column to boolean using CAST
ALTER TABLE public.servicesaddon
  ALTER COLUMN diabetes TYPE bool USING CAST((diabetes != 'No') AS bool),
  ALTER COLUMN diabetes SET DEFAULT FALSE;

-- Rename the column 'hignblood' to 'highblood'
ALTER TABLE public.patient
  RENAME COLUMN hignblood TO highblood;

-- Convert highblood column to boolean using CAST
ALTER TABLE public.patient
  ALTER COLUMN highblood TYPE bool USING CAST((highblood != 'No') AS bool),
  -- Set default value of highblood column to FALSE
  ALTER COLUMN highblood SET DEFAULT FALSE;

-- Convert hyperlipidemia column to boolean using CAST
ALTER TABLE public.servicesaddon
  ALTER COLUMN hyperlipidemia TYPE bool USING CAST((hyperlipidemia != 'No') AS bool),
  ALTER COLUMN hyperlipidemia SET DEFAULT FALSE;

-- Convert overweight column to boolean using CAST
ALTER TABLE public.servicesaddon
  ALTER COLUMN overweight TYPE bool USING CAST((overweight != 'No') AS bool),
  ALTER COLUMN overweight SET DEFAULT FALSE;

-- NHANES dataset refers to patients over 80 as 80 so doing the same for WGU patients
UPDATE public.patient SET age = 80 WHERE age > 80;

-- Adding a source column to the wgu table for comparison with nhs data
ALTER TABLE public.patient ADD COLUMN source text;
UPDATE public.patient SET source = 'WGU'; 


-- Using Python I found the list of all column names in the dataset, then renamed the significant ones for this analysis
CREATE TABLE public.nhs_demographic (
    SEQN TEXT, SDDSRVYR TEXT, RIDSTATR TEXT, gender TEXT, age FLOAT,
    RIDAGEMN TEXT, RIDRETH1 TEXT, RIDRETH3 TEXT, RIDEXMON TEXT, RIDEXAGM TEXT,
    DMQMILIZ TEXT, DMQADFC TEXT, DMDBORN4 TEXT, DMDCITZN TEXT, DMDYRSUS TEXT,
    DMDEDUC3 TEXT, DMDEDUC2 TEXT, DMDMARTL TEXT, RIDEXPRG TEXT, SIALANG TEXT,
    SIAPROXY TEXT, SIAINTRP TEXT, FIALANG TEXT, FIAPROXY TEXT, FIAINTRP TEXT,
    MIALANG TEXT, MIAPROXY TEXT, MIAINTRP TEXT, AIALANGA TEXT, DMDHHSIZ TEXT,
    DMDFMSIZ TEXT, young_children NUMERIC, old_children NUMERIC, DMDHHSZE TEXT,
    DMDHRGND TEXT, DMDHRAGZ TEXT, DMDHREDZ TEXT, DMDHRMAZ TEXT, DMDHSEDZ TEXT,
    WTINT2YR TEXT, WTMEC2YR TEXT, SDMVPSU TEXT, SDMVSTRA TEXT, INDHHIN2 TEXT,
    INDFMIN2 TEXT, INDFMPIR TEXT
);

-- Adding the seqn column as the unique key for each patient
ALTER TABLE public.nhs_demographic
ADD CONSTRAINT patient_id PRIMARY KEY (seqn);

ALTER TABLE public.nhs_demographic
 OWNER TO postgres; 

-- Import the data into demographic table
COPY nhs_demographic
FROM 'C:\Users\Public\Downloads\demographics.csv'
DELIMITER ','
CSV HEADER; 

-- Same thing for questionnare table, outputted the column names in python from the csv
CREATE TABLE public.nhs_questionnaire (
  SEQN TEXT PRIMARY KEY,
  ACD011A TEXT, ACD011B TEXT, ACD011C TEXT, ACD040 TEXT,
  ACD110 TEXT, ALQ111 TEXT, ALQ121 TEXT, ALQ130 TEXT, ALQ142 TEXT,
  ALQ270 TEXT, ALQ280 TEXT, ALQ290 TEXT, ALQ151 TEXT, ALQ170 TEXT,
  AUQ054 TEXT, AUQ060 TEXT, AUQ070 TEXT, AUQ080 TEXT, AUQ090 TEXT,
  AUQ400 TEXT, AUQ410A TEXT, AUQ410B TEXT, AUQ410C TEXT, AUQ410D TEXT,
  AUQ410E TEXT, AUQ410F TEXT, AUQ410G TEXT, AUQ410H TEXT, AUQ410I TEXT,
  AUQ410J TEXT, AUQ156 TEXT, AUQ420 TEXT, AUQ430 TEXT, AUQ139 TEXT,
  AUQ144 TEXT, AUQ147 TEXT, AUQ149A TEXT, AUQ149B TEXT, AUQ149C TEXT,
  AUQ153 TEXT, AUQ630 TEXT, AUQ440 TEXT, AUQ450A TEXT, AUQ450B TEXT,
  AUQ450C TEXT, AUQ450D TEXT, AUQ450E TEXT, AUQ450F TEXT, AUQ460 TEXT,
  AUQ470 TEXT, AUQ101 TEXT, AUQ110 TEXT, AUQ480 TEXT, AUQ490 TEXT,
  AUQ191 TEXT, AUQ250 TEXT, AUQ255 TEXT, AUQ260 TEXT, AUQ270 TEXT,
  AUQ280 TEXT, AUQ500 TEXT, AUQ300 TEXT, AUQ310 TEXT, AUQ320 TEXT,
  AUQ330 TEXT, AUQ340 TEXT, AUQ350 TEXT, AUQ360 TEXT, AUQ370 TEXT,
  AUQ510 TEXT, AUQ380 TEXT, BPQ020 TEXT, BPQ030 TEXT, BPD035 TEXT,
  BPQ040A TEXT, BPQ050A TEXT, BPQ080 TEXT, BPQ060 TEXT, BPQ070 TEXT,
  BPQ090D TEXT, BPQ100D TEXT, CDQ001 TEXT, CDQ002 TEXT, CDQ003 TEXT,
  CDQ004 TEXT, CDQ005 TEXT, CDQ006 TEXT, CDQ009A TEXT, CDQ009B TEXT,
  CDQ009C TEXT, CDQ009D TEXT, CDQ009E TEXT, CDQ009F TEXT, CDQ009G TEXT,
  CDQ009H TEXT, CDQ008 TEXT, CDQ010 TEXT, CBD071 TEXT, CBD091 TEXT,
  CBD111 TEXT, CBD121 TEXT, CBD131 TEXT, CBQ502 TEXT, CBQ503 TEXT,
  CBQ506 TEXT, CBQ536 TEXT, CBQ541 TEXT, CBQ551 TEXT, CBQ581 TEXT,
  CBQ586 TEXT, CBQ830 TEXT, CBQ835 TEXT, CBQ840 TEXT, CBQ845 TEXT,
  CBQ850 TEXT, CBQ855 TEXT, CBQ860 TEXT, CBQ865 TEXT, CBQ870 TEXT,
  CBQ875 TEXT, CBQ880 TEXT, CBQ885 TEXT, CBQ890 TEXT, CBQ895 TEXT,
  CBQ900 TEXT, CBQ645 TEXT, CBQ700 TEXT, DBQ780 TEXT, DBQ750 TEXT,
  DBQ760 TEXT, DBQ770 TEXT, CBQ905 TEXT, CBQ910 TEXT, CBQ685 TEXT,
  CBQ915 TEXT, CBQ925 TEXT, CBQ930 TEXT, CBQ935 TEXT, CBQ945 TEXT,
  CBQ950 TEXT, CBQ738A TEXT, CBQ738B TEXT, CBQ738C TEXT, CBQ738D TEXT,
  CBQ738E TEXT, CBQ738F TEXT, CBQ738G TEXT, CBQ738H TEXT, CBQ738I TEXT,
  CBQ738J TEXT, CBQ738K TEXT, CBQ738CD TEXT, CBQ698A TEXT, CBQ698B TEXT,
  CBQ698C TEXT, CBQ698D TEXT, CBQ698E TEXT, CBQ698F TEXT, CBQ698G TEXT,
  CBQ698H TEXT, CBQ698I TEXT, CBQ698CD TEXT, CBQ695A TEXT, CBQ695B TEXT,
  CBQ695C TEXT, CBQ785 TEXT, WTDRD1 TEXT, DBD930 TEXT, DBD935 TEXT,
  DBD940 TEXT, DBD945 TEXT, CBD760 TEXT, CBD765 TEXT, CBD770 TEXT,
  HSD010 TEXT, HSQ500 TEXT, HSQ510 TEXT, HSQ520 TEXT, HSQ571 TEXT,
  HSQ580 TEXT, HSQ590 TEXT, HSAQUEX TEXT, DED031 TEXT, DEQ034A TEXT,
  DEQ034C TEXT, DEQ034D TEXT, DEQ038G TEXT, DEQ038Q TEXT, DED120 TEXT,
  DED125 TEXT, DIQ010 TEXT, DID040 TEXT, DIQ160 TEXT, DIQ170 TEXT,
  DIQ172 TEXT, DIQ175A TEXT, DIQ175B TEXT, DIQ175C TEXT, DIQ175D TEXT,
  DIQ175E TEXT, DIQ175F TEXT, DIQ175G TEXT, DIQ175H TEXT, DIQ175I TEXT,
  DIQ175J TEXT, DIQ175K TEXT, DIQ175L TEXT, DIQ175M TEXT, DIQ175N TEXT,
  DIQ175O TEXT, DIQ175P TEXT, DIQ175Q TEXT, DIQ175R TEXT, DIQ175S TEXT,
  DIQ175T TEXT, DIQ175U TEXT, DIQ175V TEXT, DIQ175W TEXT, DIQ175X TEXT,
  DIQ180 TEXT, DIQ050 TEXT, DID060 TEXT, DIQ060U TEXT, DIQ070 TEXT,
  DIQ230 TEXT, DIQ240 TEXT, DID250 TEXT, DID260 TEXT, DIQ260U TEXT,
  DIQ275 TEXT, DIQ280 TEXT, DIQ291 TEXT, DIQ300S TEXT, DIQ300D TEXT,
  DID310S TEXT, DID310D TEXT, DID320 TEXT, DID330 TEXT, DID341 TEXT,
  DID350 TEXT, DIQ350U TEXT, DIQ360 TEXT, DIQ080 TEXT, DBQ010 TEXT,
  DBD030 TEXT, DBD041 TEXT, DBD050 TEXT, DBD055 TEXT, DBD061 TEXT,
  DBQ073A TEXT, DBQ073B TEXT, DBQ073C TEXT, DBQ073D TEXT, DBQ073E TEXT,
  DBQ073U TEXT, DBQ700 TEXT, DBQ197 TEXT, DBQ223A TEXT, DBQ223B TEXT,
  DBQ223C TEXT, DBQ223D TEXT, DBQ223E TEXT, DBQ223U TEXT, DBQ229 TEXT,
  DBQ235A TEXT, DBQ235B TEXT, DBQ235C TEXT, DBQ301 TEXT, DBQ330 TEXT,
  DBQ360 TEXT, DBQ370 TEXT, DBD381 TEXT, DBQ390 TEXT, DBQ400 TEXT,
  DBD411 TEXT, DBQ421 TEXT, DBQ424 TEXT, DBD895 TEXT, DBD900 TEXT,
  DBD905 TEXT, DBD910 TEXT, CBQ596 TEXT, CBQ606 TEXT, CBQ611 TEXT,
  DBQ930 TEXT, DBQ935 TEXT, DBQ940 TEXT, DBQ945 TEXT, DLQ010 TEXT,
  DLQ020 TEXT, DLQ040 TEXT, DLQ050 TEXT, DLQ060 TEXT, DLQ080 TEXT,
  DLQ100 TEXT, DLQ110 TEXT, DLQ130 TEXT, DLQ140 TEXT, DLQ150 TEXT,
  DLQ170 TEXT, DUQ200 TEXT, DUQ210 TEXT, DUQ211 TEXT, DUQ213 TEXT,
  DUQ215Q TEXT, DUQ215U TEXT, DUQ217 TEXT, DUQ219 TEXT, DUQ220Q TEXT,
  DUQ220U TEXT, DUQ230 TEXT, DUQ240 TEXT, DUQ250 TEXT, DUQ260 TEXT,
  DUQ270Q TEXT, DUQ270U TEXT, DUQ272 TEXT, DUQ280 TEXT, DUQ290 TEXT,
  DUQ300 TEXT, DUQ310Q TEXT, DUQ310U TEXT, DUQ320 TEXT, DUQ330 TEXT,
  DUQ340 TEXT, DUQ350Q TEXT, DUQ350U TEXT, DUQ352 TEXT, DUQ360 TEXT,
  DUQ370 TEXT, DUQ380A TEXT, DUQ380B TEXT, DUQ380C TEXT, DUD380F TEXT,
  DUQ390 TEXT, DUQ400Q TEXT, DUQ400U TEXT, DUQ410 TEXT, DUQ420 TEXT,
  DUQ430 TEXT, ECD010 TEXT, ECQ020 TEXT, ECD070A TEXT, ECD070B TEXT,
  ECQ080 TEXT, ECQ090 TEXT, WHQ030E TEXT, MCQ080E TEXT, ECQ150 TEXT,
  FSD032A TEXT, FSD032B TEXT, FSD032C TEXT, FSD041 TEXT, FSD052 TEXT,
  FSD061 TEXT, FSD071 TEXT, FSD081 TEXT, FSD092 TEXT, FSD102 TEXT,
  FSD032D TEXT, FSD032E TEXT, FSD032F TEXT, FSD111 TEXT, FSD122 TEXT,
  FSD132 TEXT, FSD141 TEXT, FSD146 TEXT, FSDHH TEXT, FSDAD TEXT,
  FSDCH TEXT, FSD151 TEXT, FSQ165 TEXT, FSD165N TEXT, FSQ012 TEXT,
  FSD012N TEXT, FSD230 TEXT, FSD230N TEXT, FSD795 TEXT, FSD225 TEXT,
  FSD235 TEXT, FSD855 TEXT, FSD860 TEXT, FSQ865 TEXT, FSQ162 TEXT,
  FSQ760 TEXT, FSD760N TEXT, FSQ653 TEXT, FSD660ZC TEXT, FSD675 TEXT,
  FSD680 TEXT, FSD670ZC TEXT, FSQ690 TEXT, FSQ695 TEXT, FSD652ZW TEXT,
  FSD672ZW TEXT, FSD652CW TEXT, FSD660ZW TEXT, HIQ011 TEXT, HIQ031A TEXT,
  HIQ031B TEXT, HIQ031C TEXT, HIQ031D TEXT, HIQ031E TEXT, HIQ031F TEXT,
  HIQ031H TEXT, HIQ031I TEXT, HIQ031J TEXT, HIQ031AA TEXT, HIQ260 TEXT,
  HIQ105 TEXT, HIQ270 TEXT, HIQ210 TEXT, HEQ010 TEXT, HEQ020 TEXT,
  HEQ030 TEXT, HEQ040 TEXT, HUQ010 TEXT, HUQ020 TEXT, HUQ030 TEXT,
  HUQ041 TEXT, HUQ051 TEXT, HUQ061 TEXT, HUQ071 TEXT, HUD080 TEXT,
  HUQ090 TEXT, HOD050 TEXT, HOQ065 TEXT, IMQ011 TEXT, IMQ020 TEXT,
  IMQ060 TEXT, IMQ070 TEXT, IMQ081A TEXT, IMQ081B TEXT, IMQ081C TEXT,
  IMQ081D TEXT, IMQ090 TEXT, IMQ100 TEXT, INQ020 TEXT, INQ012 TEXT,
  INQ030 TEXT, INQ060 TEXT, INQ080 TEXT, INQ090 TEXT, INQ132 TEXT,
  INQ140 TEXT, INQ150 TEXT, IND235 TEXT, INDFMMPI TEXT, INDFMMPC TEXT,
  INQ300 TEXT, IND310 TEXT, INQ320 TEXT, KIQ022 TEXT, KIQ025 TEXT,
  KIQ026 TEXT, KIQ029 TEXT, KIQ005 TEXT, KIQ010 TEXT, KIQ042 TEXT,
  KIQ430 TEXT, KIQ044 TEXT, KIQ450 TEXT, KIQ046 TEXT, KIQ470 TEXT,
  KIQ050 TEXT, KIQ052 TEXT, KIQ480 TEXT, MCQ010 TEXT, MCQ025 TEXT,
  MCQ035 TEXT, MCQ040 TEXT, MCQ050 TEXT, AGQ030 TEXT, MCQ053 TEXT,
  MCQ080 TEXT, MCQ092 TEXT, MCD093 TEXT, MCQ149 TEXT, MCQ151 TEXT,
  RHD018 TEXT, MCQ160A TEXT, MCD180A TEXT, MCQ195 TEXT, MCQ160N TEXT,
  MCD180N TEXT, MCQ160B TEXT, MCD180B TEXT, MCQ160C TEXT, MCD180C TEXT,
  MCQ160D TEXT, MCD180D TEXT, MCQ160E TEXT, MCD180E TEXT, MCQ160F TEXT,
  MCD180F TEXT, MCQ160M TEXT, MCQ170M TEXT, MCD180M TEXT, MCQ160G TEXT,
  MCD180G TEXT, MCQ160K TEXT, MCQ170K TEXT, MCD180K TEXT, MCQ160O TEXT,
  MCQ160L TEXT, MCQ170L TEXT, MCD180L TEXT, MCQ500 TEXT, MCQ510A TEXT,
  MCQ510B TEXT, MCQ510C TEXT, MCQ510D TEXT, MCQ510E TEXT, MCQ510F TEXT,
  MCQ520 TEXT, MCQ530 TEXT, MCQ540 TEXT, MCQ550 TEXT, MCQ560 TEXT,
  MCQ570 TEXT, MCQ203 TEXT, MCQ206 TEXT, MCQ220 TEXT, MCQ230A TEXT,
  MCD240A TEXT, MCQ230B TEXT, MCD240B TEXT, MCQ230C TEXT, MCD240C TEXT,
  MCQ230D TEXT, MCQ300B TEXT, MCQ300C TEXT, MCQ300A TEXT, MCQ366A TEXT,
  MCQ366B TEXT, MCQ366C TEXT, MCQ366D TEXT, MCQ371A TEXT, MCQ371B TEXT,
  MCQ371C TEXT, MCQ371D TEXT, OSQ230 TEXT, DPQ010 TEXT, DPQ020 TEXT,
  DPQ030 TEXT, DPQ040 TEXT, DPQ050 TEXT, DPQ060 TEXT, DPQ070 TEXT,
  DPQ080 TEXT, DPQ090 TEXT, DPQ100 TEXT, OCD150 TEXT, OCQ180 TEXT,
  OCQ210 TEXT, OCQ260 TEXT, OCD270 TEXT, OCQ670 TEXT, OCQ380 TEXT,
  OCD390G TEXT, OCD395 TEXT, OHQ030 TEXT, OHQ033 TEXT, OHQ770 TEXT,
  OHQ780A TEXT, OHQ780B TEXT, OHQ780C TEXT, OHQ780D TEXT, OHQ780E TEXT,
  OHQ780F TEXT, OHQ780G TEXT, OHQ780H TEXT, OHQ780I TEXT, OHQ780J TEXT,
  OHQ780K TEXT, OHQ555G TEXT, OHQ555Q TEXT, OHQ555U TEXT, OHQ560G TEXT,
  OHQ560Q TEXT, OHQ560U TEXT, OHQ566 TEXT, OHQ571Q TEXT, OHQ571U TEXT,
  OHQ576G TEXT, OHQ576Q TEXT, OHQ576U TEXT, OHQ610 TEXT, OHQ612 TEXT,
  OHQ614 TEXT, OHQ620 TEXT, OHQ640 TEXT, OHQ680 TEXT, OHQ835 TEXT,
  OHQ845 TEXT, OHQ848G TEXT, OHQ848Q TEXT, OHQ849 TEXT, OHQ850 TEXT,
  OHQ860 TEXT, OHQ870 TEXT, OHQ880 TEXT, OHQ895 TEXT, OHQ900 TEXT,
  OSQ010A TEXT, OSQ010B TEXT, OSQ010C TEXT, OSQ020A TEXT, OSQ020B TEXT,
  OSQ020C TEXT, OSD030AA TEXT, OSQ040AA TEXT, OSD050AA TEXT, OSD030AB TEXT,
  OSQ040AB TEXT, OSD050AB TEXT, OSD030AC TEXT, OSQ040AC TEXT, OSD050AC TEXT,
  OSD030BA TEXT, OSQ040BA TEXT, OSD050BA TEXT, OSD030BB TEXT, OSQ040BB TEXT,
  OSD050BB TEXT, OSD030BC TEXT, OSQ040BC TEXT, OSD050BC TEXT, OSD030BD TEXT,
  OSQ040BD TEXT, OSD050BD TEXT, OSD030BE TEXT, OSQ040BE TEXT, OSD050BE TEXT,
  OSD030CA TEXT, OSQ040CA TEXT, OSD050CA TEXT, OSD030CB TEXT, OSQ040CB TEXT,
  OSD050CB TEXT, OSD030CC TEXT, OSQ040CC TEXT, OSD050CC TEXT, OSD030CD TEXT,
  OSQ040CD TEXT, OSD050CD TEXT, OSD030CE TEXT, OSQ040CE TEXT, OSD050CE TEXT,
  OSQ080 TEXT, OSQ090A TEXT, OSQ100A TEXT, OSD110A TEXT, OSQ120A TEXT,
  OSQ090B TEXT, OSQ100B TEXT, OSD110B TEXT, OSQ120B TEXT, OSQ090C TEXT,
  OSQ100C TEXT, OSD110C TEXT, OSQ120C TEXT, OSQ090D TEXT, OSQ100D TEXT,
  OSD110D TEXT, OSQ120D TEXT, OSQ090E TEXT, OSQ100E TEXT, OSD110E TEXT,
  OSQ120E TEXT, OSQ090F TEXT, OSQ100F TEXT, OSD110F TEXT, OSQ120F TEXT,
  OSQ090G TEXT, OSQ120G TEXT, OSQ090H TEXT, OSQ120H TEXT, OSQ090I TEXT,
  OSQ120I TEXT, OSQ090J TEXT, OSQ100J TEXT, OSD110J TEXT, OSQ120J TEXT,
  OSQ060 TEXT, OSQ072 TEXT, OSQ130 TEXT, OSQ140Q TEXT, OSQ140U TEXT,
  OSQ150 TEXT, OSQ160A TEXT, OSQ160B TEXT, OSQ170 TEXT, OSQ180 TEXT,
  OSQ190 TEXT, OSQ200 TEXT, OSQ210 TEXT, OSQ220 TEXT, PUQ100 TEXT,
  PUQ110 TEXT, PAQ605 TEXT, PAQ610 TEXT, PAD615 TEXT, PAQ620 TEXT,
  PAQ625 TEXT, PAD630 TEXT, PAQ635 TEXT, PAQ640 TEXT, PAD645 TEXT,
  PAQ650 TEXT, PAQ655 TEXT, PAD660 TEXT, PAQ665 TEXT, PAQ670 TEXT,
  PAD675 TEXT, PAD680 TEXT, PAQ706 TEXT, PAQ710 TEXT, PAQ715 TEXT,
  PFQ020 TEXT, PFQ030 TEXT, PFQ033 TEXT, PFQ041 TEXT, PFQ049 TEXT,
  PFQ051 TEXT, PFQ054 TEXT, PFQ057 TEXT, PFQ059 TEXT, PFQ061A TEXT,
  PFQ061B TEXT, PFQ061C TEXT, PFQ061D TEXT, PFQ061E TEXT, PFQ061F TEXT,
  PFQ061G TEXT, PFQ061H TEXT, PFQ061I TEXT, PFQ061J TEXT, PFQ061K TEXT,
  PFQ061L TEXT, PFQ061M TEXT, PFQ061N TEXT, PFQ061O TEXT, PFQ061P TEXT,
  PFQ061Q TEXT, PFQ061R TEXT, PFQ061S TEXT, PFQ061T TEXT, PFQ063A TEXT,
  PFQ063B TEXT, PFQ063C TEXT, PFQ063D TEXT, PFQ063E TEXT, PFQ090 TEXT,
  RXQ510 TEXT, RXQ515 TEXT, RXQ520 TEXT, RXQ525G TEXT, RXQ525Q TEXT,
  RXQ525U TEXT, RXD530 TEXT, RHQ010 TEXT, RHQ020 TEXT, RHQ031 TEXT,
  RHD043 TEXT, RHQ060 TEXT, RHQ070 TEXT, RHQ074 TEXT, RHQ076 TEXT,
  RHQ078 TEXT, RHQ131 TEXT, RHD143 TEXT, RHQ160 TEXT, RHQ162 TEXT,
  RHQ163 TEXT, RHQ166 TEXT, RHQ169 TEXT, RHQ172 TEXT, RHD173 TEXT,
  RHQ171 TEXT, RHD180 TEXT, RHD190 TEXT, RHQ197 TEXT, RHQ200 TEXT,
  RHD280 TEXT, RHQ291 TEXT, RHQ305 TEXT, RHQ332 TEXT, RHQ420 TEXT,
  RHQ540 TEXT, RHQ542A TEXT, RHQ542B TEXT, RHQ542C TEXT, RHQ542D TEXT,
  RHQ554 TEXT, RHQ560Q TEXT, RHQ560U TEXT, RHQ570 TEXT, RHQ576Q TEXT,
  RHQ576U TEXT, RHQ580 TEXT, RHQ586Q TEXT, RHQ586U TEXT, RHQ596 TEXT,
  RHQ602Q TEXT, RHQ602U TEXT, SLQ300 TEXT, SLQ310 TEXT, SLD012 TEXT,
  SLQ320 TEXT, SLQ330 TEXT, SLD013 TEXT, SLQ030 TEXT, SLQ040 TEXT,
  SLQ050 TEXT, SLQ120 TEXT, SMQ020 TEXT, SMD030 TEXT, SMQ040 TEXT,
  SMQ050Q TEXT, SMQ050U TEXT, SMD057 TEXT, SMQ078 TEXT, SMD641 TEXT,
  SMD650 TEXT, SMD093 TEXT, SMDUPCA TEXT, SMD100BR TEXT, SMD100FL TEXT,
  SMD100MN TEXT, SMD100LN TEXT, SMD100TR TEXT, SMD100NI TEXT, SMD100CO TEXT,
  SMQ621 TEXT, SMD630 TEXT, SMQ661 TEXT, SMQ665A TEXT, SMQ665B TEXT,
  SMQ665C TEXT, SMQ665D TEXT, SMQ670 TEXT, SMQ848 TEXT, SMQ852Q TEXT,
  SMQ852U TEXT, SMQ890 TEXT, SMQ895 TEXT, SMQ900 TEXT, SMQ905 TEXT,
  SMQ910 TEXT, SMAQUEX2 TEXT, SMD460 TEXT, SMD470 TEXT, SMD480 TEXT,
  SMQ681 TEXT, SMQ690A TEXT, SMQ710 TEXT, SMQ720 TEXT, SMQ725 TEXT,
  SMQ690B TEXT, SMQ740 TEXT, SMQ690C TEXT, SMQ770 TEXT, SMQ690G TEXT,
  SMQ845 TEXT, SMQ690H TEXT, SMQ849 TEXT, SMQ851 TEXT, SMQ690D TEXT,
  SMQ800 TEXT, SMQ690E TEXT, SMQ817 TEXT, SMQ690K TEXT, SMQ857O TEXT,
  SMQ863 TEXT, SMQ690F TEXT, SMQ830 TEXT, SMQ840 TEXT, SMDANY TEXT,
  SMAQUEX TEXT, SMQ856 TEXT, SMQ858 TEXT, SMQ860 TEXT, SMQ862 TEXT,
  SMQ866 TEXT, SMQ868 TEXT, SMQ870 TEXT, SMQ872 TEXT, SMQ874 TEXT,
  SMQ876 TEXT, SMQ878 TEXT, SMQ880 TEXT, SMQ940 TEXT, WTSVOC2Y TEXT,
  VTQ210 TEXT, VTQ200A TEXT, VTQ231A TEXT, VTQ233A TEXT, VTQ233B TEXT,
  VTQ241A TEXT, VTD241B TEXT, VTQ244A TEXT, VTD244B TEXT, VTQ251A TEXT,
  VTD251B TEXT, VTQ261A TEXT, VTD261B TEXT, VTQ271A TEXT, VTD271B TEXT,
  VTQ281A TEXT, VTD281B TEXT, VTQ281C TEXT, VTD281D TEXT, VTQ281E TEXT,
  VTD281F TEXT, WHD010 TEXT, WHD020 TEXT, WHQ030 TEXT, WHQ040 TEXT,
  WHD050 TEXT, WHQ060 TEXT, WHQ070 TEXT, WHD080A TEXT, WHD080B TEXT,
  WHD080C TEXT, WHD080D TEXT, WHD080E TEXT, WHD080F TEXT, WHD080G TEXT,
  WHD080H TEXT, WHD080I TEXT, WHD080J TEXT, WHD080K TEXT, WHD080M TEXT,
  WHD080N TEXT, WHD080O TEXT, WHD080P TEXT, WHD080Q TEXT, WHD080R TEXT,
  WHD080S TEXT, WHD080T TEXT, WHD080U TEXT, WHD080L TEXT, WHQ225 TEXT,
  WHD110 TEXT, WHD120 TEXT, WHD130 TEXT, WHD140 TEXT, WHQ150 TEXT,
  WHQ190 TEXT, WHQ200 TEXT, WHQ030M TEXT, WHQ500 TEXT, WHQ520 TEXT, test TEXT
);

-- import the data into questionnare table
COPY nhs_questionnaire
FROM 'C:\Users\Public\Downloads\questionnaire.csv'
DELIMITER ','
CSV HEADER; 

-- Renaming columns into a readable format for analysis
ALTER TABLE public.nhs_questionnaire
RENAME COLUMN MCQ160A TO arthritis;

ALTER TABLE public.nhs_questionnaire
RENAME COLUMN BPQ080 TO hyperlipidemia;

ALTER TABLE public.nhs_questionnaire
RENAME COLUMN MCQ080 TO overweight;

ALTER TABLE public.nhs_questionnaire
RENAME COLUMN DIQ010 TO diabetes;

ALTER TABLE public.nhs_questionnaire
RENAME COLUMN BPQ020 TO highblood;

ALTER TABLE public.nhs_questionnaire
 OWNER TO postgres; 

--convert gender column to booleans using CAST
UPDATE public.nhs_demographic 
SET gender = CAST(
    CASE 
        WHEN gender = '1' THEN 'Male'
        WHEN gender = '2' THEN 'Female'
    END AS TEXT
);

-- convert highblood column to boolean using CAST
ALTER TABLE public.nhs_questionnaire 
ALTER COLUMN highblood TYPE bool 
USING CAST(CASE WHEN highblood = '1' THEN TRUE ELSE FALSE END AS BOOLEAN);

-- convert overweight column to boolean using CAST
ALTER TABLE public.nhs_questionnaire 
ALTER COLUMN overweight TYPE bool 
USING CAST(CASE WHEN overweight = '1' THEN TRUE ELSE FALSE END AS BOOLEAN);

-- convert arthritis column to boolean using CAST
ALTER TABLE public.nhs_questionnaire 
ALTER COLUMN arthritis TYPE bool 
USING CAST(CASE WHEN arthritis = '1' THEN TRUE ELSE FALSE END AS BOOLEAN);

-- convert diabetes column to boolean using CAST
ALTER TABLE public.nhs_questionnaire 
ALTER COLUMN diabetes TYPE bool 
USING CAST(CASE WHEN diabetes = '1' THEN TRUE ELSE FALSE END AS BOOLEAN);

-- convert hyperlipidemia column to boolean using CAST
ALTER TABLE public.nhs_questionnaire 
ALTER COLUMN hyperlipidemia TYPE bool 
USING CAST(CASE WHEN hyperlipidemia = '1' THEN TRUE ELSE FALSE END AS BOOLEAN);

-- Adding a source column to the nhs table for comparison with wgu data
ALTER TABLE public.nhs_demographic ADD COLUMN source text;
UPDATE public.nhs_demographic SET source = 'NHS'; 

-- The WGU dataset didnt include minors, but NHS results did so need to remove minors for comparison
DELETE FROM nhs_questionnaire 
USING nhs_demographic 
WHERE nhs_questionnaire.seqn = nhs_demographic.seqn 
  AND nhs_demographic.age < 18;

DELETE FROM nhs_demographic 
WHERE age < 18;

-- For referential integrity, adding seqn foreign key to nhs questionnare table
ALTER TABLE public.nhs_questionnaire 
ADD FOREIGN KEY (seqn) REFERENCES public.nhs_demographic(seqn);

-- Noticed wgu tables didnt have the foreign key added to it, adding patient_id as foreign key
ALTER TABLE public.servicesaddon 
ADD FOREIGN KEY (patient_id) REFERENCES public.patient(patient_id);

-- Noticed wgu tables didnt have the foreign key added to it, adding patient_id as foreign key
ALTER TABLE public.survey_responses_addon 
ADD FOREIGN KEY (patient_id) REFERENCES public.patient(patient_id);

-- create a new table by unioning all the columns we're interested in from all 4 tables
-- selects columns from wgu patient/servicesaddon and nhs demographics/questionnare tables
CREATE TABLE wgu_nhs AS (
    SELECT 
        wgu.age, 
        wgu.gender, 
        wgu.children, 
        condition.arthritis, 
        condition.diabetes,
        wgu.highblood, 
        condition.hyperlipidemia, 
        condition.overweight, 
        wgu.source
    FROM patient AS wgu
    INNER JOIN servicesaddon AS condition
    ON wgu.patient_id = condition.patient_id
    
    UNION ALL
    
    SELECT 
        nhs.age, 
        nhs.gender, 
        nhs.young_children + nhs.old_children AS children, 
        question.arthritis, 
        question.diabetes, 
        question.highblood,
        question.hyperlipidemia,
        question.overweight, 
        nhs.source
    FROM nhs_demographic AS nhs
    INNER JOIN nhs_questionnaire AS question
    ON nhs.seqn = question.seqn
);
```

## B. Dashboard Presentation - Panopto<a id="panopto"></a>

https://wgu.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=cfa616d1-51d6-4707-a875-b1820070487a

## C. Dashboard Purpose<a id="dashboard-purpose"></a>

In previous courses, the WGU medical dataset was used to analyze readmission rates among hospital patients. This dashboard does not directly analyze readmissions but instead examines specific trends and demographics within both datasets. By taking a broader approach and analyzing medical condition rates and demographic trends among patients, it provides a contextualized view of the dataset while highlighting additional insights. These insights can lead to initiatives that improve patient care and ideally reduce overall readmission rates for WGU's hospitals.

As mentioned in the previous course, and for the purpose of this analysis, WGU conducted a study with patients across the country. The patient demographics tab on the dashboard demonstrates some of the differences and similarities between the WGU study and the NHANES study among patient demographics. By providing visualizations of gender, age, and children for each dataset, WGU can analyze trends among specific demographics for their hospital research. For example, the NHANES study shows a lower count of older patients compared to the WGU patients. This may indicate a need to focus additional resources on caring for older patients based on this analysis. Stakeholders have a vested interest in improving the care of their patients and these insights can lead to actionable initiatives for improving the patient quality of life.

The condition metrics tab on this dashboard displays critical patient health conditions. By visualizing the breakdown of significant health conditions, leaders at WGU's hospitals can gain important insights into where to allocate resources. Patients who are overweight, have diabetes, or high blood pressure are displayed in 5-year age groups. Key Performance Indicators (KPIs) are shown to the right of each condition heatmap to indicate the rates of each disease for each dataset. For example, this tab shows that WGU's patients are diagnosed with diabetes at a much higher rate compared to NHANES patients. This context can lead to increased screenings and preventative care for these particular patients.

### C2. Dashboard Justification<a id="dashboard-justification"></a>

Tableau Desktop was the business intelligence tool used for this analysis. Tableau provides an intuitive way to create complex, interactive visualizations. These visualizations help communicate analysis insights, uncover new relationships within datasets, and reveal interesting observations. When presented visually, these analysis techniques become easily digestible to a wider audience. Tableau offers a seamless way to create presentations of these analyses, which can be quickly shared with various groups.

### C3. Data Preparation<a id="data-preparation"></a>

Before Tableau can be used, the data needed to be prepared for this analysis, the following steps outline the modification and creations performed on both the WGU and NHANES datasets:

- As shown in previous courses, the WGU patient gender column has a Non-Binary option, the column was modified to categorize patients as Non-Binary if they selected "Prefer not to answer".
- Using CAST in SQL, the patient conditions needed to be converted from Yes/No to booleans for easier analysis.
- The NHANES dataset refers to patients over 80 as 80, the WGU dataset was modified to match this condition as well.
- To differentiate between the two datasets, a source column was added to both tables, "WGU" and "NHS" for distinction.
- The NHANES datasets were split into nhs_demographics and nhs_questionnare tables, each having many unused columns. The columns used in this analysis were renamed to provide clarity after importing the data from the CSV files. The CDC data dictionary was used to identify columns of interest for this analysis.
- The NHANES gender column was converted to a string format to match the WGU dataset.
- Using CAST in SQL the condition columns for NHS patients were converted to booleans.
- Since the WGU dataset did not include minors but the NHANES dataset did, the minors were removed from the NHANES dataset.
- A primary key seqn was added to the new nhs_demographic table. Foreign keys were added to the servicesaddon and survey_responses_addon tables for the patient_id column to provide referential integrity. Same for the seqn column in the nhs_questionnare table.
- Once the new NHANES tables were generated and the WGU data was modified, a new table wgu_nhs was created as a UNION between the wgu patient/condition tables and the NHANES demographic/questionnare tables.

### C4. Dashboard Creation<a id="dashboard-creation"></a>

For creating the worksheets and dashboards within this analysis the following are a list of steps taken to produce these resources:

Patient Genders Worksheet:
- Select Source field and drag it to the columns area
- Select Gender field and drag it to the color area in the Marks section
- Also drag Gender to the angle area in the Marks section, then right-click Gender and select Measure -> Count
- Double click the title “Sheet 1” and change it to “Patient Gender Breakdown”, use Tableau Bold and center the title
- Click the Label tile in the Marks area and tick “Show mark labels”
- Click the Color tile in the Marks area, click Edit Colors, and select the Color Blind palette in the dropdown, click Assign Palette and Apply + OK.
- Click the Tooltip tile in the Marks area, change the text to: <CNT(Gender)> <Source> Patients are <Gender>
- In the Tables area, right click Gender and select “Show Filter”, in the Filters area right click Gender and select Apply to Worksheets -> All Using This Data Source
- In the top menu where it says “Standard” change it to “Fit Width”
- Right click the Source header in the chart view and click “Hide Field Labels for Columns”

Patient Ages Worksheet:
- In the Tables area, right-click Ages and click Create -> Bins -> Size of bins = 5, click OK.
- Select Source and drag it to the columns area
- Select Age (bin) and drag it to the columns area to the right of Source
- in Tables section, Right click Age and select Change Data Type -> Number (whole) then drag it to the rows area, right click and select Measure -> Count
- In the top menu where it says “Standard” change it to “Fit Width”
- Right click the Source / Age (bin) header in the graph view and click “Hide Field Labels for Columns”
- Click the Tooltip tile in the Marks area, change the text to: <CNT(Age)> <Source> Patients are <Age (bin)>
- In the Tables area, right click Age and select “Show Filter”, in the Filters area right click Age and select Dimension, and right click again and select Apply to Worksheets -> All Using This Data Source, finally select Show Filter again
- Double click the title “Sheet 2” and change it to “Count of Patients Ages (5 yr groups)”, use Tableau Bold and center the title

Patient Children Worksheet:
- In the Tables area, right-click Children and click Create -> Bins -> Size of bins = 1, click OK.
- Select Source and drag it to the columns area
- Select Children (bin) and drag it to the columns area to the right of Source
- Select Children and drag it to the rows area, right click and select Measure -> Count, then right click again and select Quick Table Calculation -> Percent of Total, finally right click again and select Edit Table Calculation, under Compute Using select Pane (across)
- Right click the Source header in the table view and click “Hide Field Labels for Columns”
- Click the Label tile in the Marks area and tick “Show mark labels”, also set the Alignment Direction to Up
- Click the Tooltip tile in the Marks area, change the text to: <% of Total CNT(Children)> of the <Source> Patients have <Children (bin)> children
- In the top menu where it says “Standard” change it to “Fit Width”
- Double click the title “Sheet 3” and change it to “Count of Patient Children”, use Tableau Bold and center the title
- Double click the axis “% of Total Count of Children” and change it to “Percent of Patients”

Condition Rates KPIs (High Blood Pressure):
- Select Source and drag it to the columns area
- Select Highblood and drag it to the rows area
- Select wgu_nhs (Count) and drag it to the Text area in the Marks section, then right-click and select Quick Table Calculation -> Percent of Total, right-click and select Edit Table Calculation, select Table (down) in the Compute Using section
- Double click the title “Sheet 4” and change it to “High BP Condition Rate”
- In the row titled “False” right click the cell and select Hide
- In the row titled “True” right click the cell and de-select Show Header
- Click the Tooltip tile in the Marks area, change the text to: <% of Total CNT(wgu_nhs)> of <Source> Patients have high blood pressure

Condition Rates KPIs (Diabetes):
- Select Source and drag it to the columns area
- Select Diabetes and drag it to the rows area
- Select wgu_nhs (Count) and drag it to the Text area in the Marks section, then right-click and select Quick Table Calculation -> Percent of Total, right-click and select Edit Table Calculation, select Table (down) in the Compute Using section
- Double click the title “Sheet 5” and change it to “Diabetes Condition Rate”
- In the row titled “False” right click the cell and select Hide
- In the row titled “True” right click the cell and de-select Show Header
- Click the Tooltip tile in the Marks area, change the text to: <% of Total CNT(wgu_nhs)> of <Source> Patients have diabetes

Condition Rates KPIs (Overweight):
- Select Source and drag it to the columns area
- Select Overweight and drag it to the rows area
- Select wgu_nhs (Count) and drag it to the Text area in the Marks section, then right-click and select Quick Table Calculation -> Percent of Total, right-click and select Edit Table Calculation, select Table (down) in the Compute Using section
- Double click the title “Sheet 6” and change it to “Overweight Patients Rate”
- In the row titled “False” right click the cell and select Hide
- In the row titled “True” right click the cell and de-select Show Header
- Click the Tooltip tile in the Marks area, change the text to: <% of Total CNT(wgu_nhs)> of <Source> Patients are overweight

High Blood Pressure By Age Worksheet:
- Select Age (bin) that we created earlier during the Patient Ages worksheet, drag it to the columns area
- Select Source and drag it to the rows area
- Select Highblood and drag it to the Filters section, select Highblood = true
- Select Highblood and drag it to the Colors area in the Marks section, right click and select Measure -> Count
- Double click the title “Sheet 7” and change it to “Count of Patients with High Blood Pressure by Age (5 yr groups)”, use Tableau Bold and center the title
- Right click the Age (bin) header in the heatmap view and click “Hide Field Labels for Columns”
- Right click the Highblood header in the heatmap view and click “Hide Field Labels for Rows”
- Click the Label tile in the Marks area and tick “Show mark labels”
- In the top menu where it says “Standard” change it to “Fit Width”

Diabetes By Age Worksheet:
- Select Age (bin) that we created earlier during the Patient Ages worksheet, drag it to the columns area
- Select Source and drag it to the rows area
- Select Diabetes and drag it to the Filters section, select Diabetes = true
- Select Diabetes and drag it to the Colors area in the Marks section, right click and select Measure -> Count
- Double click the title “Sheet 8” and change it to “Count of Patients with Diabetes by Age (5 yr groups)”, use Tableau Bold and center the title
- Right click the Age (bin) header in the heatmap view and click “Hide Field Labels for Columns”
- Right click the Diabetes header in the heatmap view and click “Hide Field Labels for Rows”
- Click the Label tile in the Marks area and tick “Show mark labels”
- In the top menu where it says “Standard” change it to “Fit Width”

Overweight By Age Worksheet:
- Select Age (bin) that we created earlier during the Patient Ages worksheet, drag it to the columns area
- Select Source and drag it to the rows area
- Select Overweight and drag it to the Filters section, select Overweight = true
- Select Overweight and drag it to the Colors area in the Marks section, right click and select Measure -> Count
- Double click the title “Sheet 9” and change it to “Count of Overweight Patients by Age (5 yr groups)”, use Tableau Bold and center the title
- Right click the Age (bin) header in the heatmap view and click “Hide Field Labels for Columns”
- Right click the Overweight header in the heatmap view and click “Hide Field Labels for Rows”
- Click the Label tile in the Marks area and tick “Show mark labels”
- In the top menu where it says “Standard” change it to “Fit Width”


Patient Demographics Dashboard:
- Select the Patient Genders worksheet and drag it to the top left corner, can set it to Floating to resize accordingly 
- Under More Options of the Patient Genders worksheet select Show Filters - select age/gender filters and align to top of dashboard
- Select the Patient Ages worksheet and drag it to the middle portion of the dashboard, select floating and resize accordingly
- Select the Patient Children worksheet and drag it to the bottom portion of the dashboard, select floating and resize accordingly
- Select the More Options of each worksheet and select Use As Filter

Condition Metrics Dashboard:
- Select the High Blood Pressure KPI and drag it to the left side of the dashboard, set it to floating and resize accordingly
- Repeat the exact same steps for Diabetes and Overweight KPIs
- Select the High Blood Pressure by Age worksheet onto the dashboard, set it to floating and align to the right side of the High BP KPI
- Repeat the exact same steps for Diabetes and Overweight by Age worksheets, aligning them next to their respective KPIs
- Remove the heatmap legends as they weren’t needed for this analysis and didn’t provide much more context
- Both the Age and Gender Filters appeared automatically on the right-side of the dashboard, if not showing, select More Options under one of the heat maps and select Show Filters -> Age and Gender and align them to the right side of the dashboard
- Select More Options within each worksheet in the dashboard and select Use As Filter
- Add a title for the dashboard: “Condition KPIs and Condition Breakdowns by Age” - “Use the filters on the right to filter by Age or Gender”, use Tableau Bold and center the title


### C5. Data Analysis Results<a id="data-analysis"></a>

The two main results taken from this analysis were the differences in patient demographics and the ratio of patients diagnosed with specific condition rates between the two datasets. The first result showed that WGU patients had a larger population of older patients. Even considering the scale difference since the NHS dataset has about 5,800 rows vs WGU with 10,000, with WGU's dataset being almost twice as large as the NHANES results, this does not account for the large difference in patient age counts. WGU's clinics seem to care for a larger number of older patients, suggesting that additional resources should be allocated to caring for these specific patients. WGU patients also had more children, with over 60% of NHANES patients having no children at all.

The second result highlighted the difference in the condition rates between the two datasets. By viewing the diabetes condition by age visualization, we see a significant increase in the number of WGU patients diagnosed with diabetes and high blood pressure compared to the NHANES dataset. Even after accounting for the dataset sizes, the number of patients with these condition rates is still much higher in the WGU clinics. This insight could lead to initiatives focused on screenings and care for these patients. Acting on these results can, in turn, lead to lower readmission rates for the WGU hospital system.


### C6. Analysis Limitations<a id="limitations"></a>

There are several limitations to this analysis that are worth mentioning. The NHANES dataset samples about 5,000 patients per year across the United States. According to the WGU data dictionary, WGU has hospitals across the US, so the results are expected to be a reliable sample nationwide. The accuracy of the dataset sample locations could influence how these results are interpreted. By using several other datasets with reliable sampling from across the country, the accuracy of these results could be increased. The WGU dataset also only records patients who have been hospitalized, whereas the NHANES results may include non-hospitalized individuals. This difference can lead to different outcomes in the data, and insights may not be entirely accurate based on this context.

The WGU dataset also had a "Prefer not to Answer" option in the gender column, which was renamed to "Non-Binary" to match previous analysis. The NHANES dataset did not account for other genders, which can lead to inaccurate results regarding medical conditions. Even if other genders are not the majority, they can still affect the data. Patient ages were also limited, as NHANES refers to patients over 80 as 80. The WGU dataset was modified to match this condition. Patients over 80 likely have increased medical issues, and analyzing this specific age range was not possible with this dataset. For analyzing elderly patients against the WGU dataset, an alternative external dataset should be used instead. The same scenario applies to WGU not accounting for patients under the age of 18. While minors may not have as many medical issues, it is still a demographic worth analyzing. In the future, WGU should make an effort to account for patients under the age of 18, as their identification is not listed in the dataset.

<a id="thirdparty"></a>
## D. Third-party Code References

SQL Server CAST() Function. (n.d.). https://www.w3schools.com/sql/func_sqlserver_cast.asp


<a id="references"></a>
## E. References

WGU Datasets. (n.d.). WGU Performance Assessment. Tasks.wgu.edu. Retrieved from
https://lrps.wgu.edu/provision/227080088

National Health & Nutrition Exam Survey 2017-2018. (2024, January 12). Kaggle.
https://www.kaggle.com/datasets/rileyzurrin/national-health-and-nutrition-exam-survey-2017-2018

NHANES 2017-2018 Demographics Variable list. (n.d.).
https://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Demographics&Cycle=2017-2018